In [1]:
from jax import vmap, jit
import pickle
import pandas as pd
import io
import yaml
from functools import partial
import jax.numpy as jnp

In [9]:
test_a = jnp.arange(5, dtype=float)

In [19]:
jnp.nansum()

140 µs ± 25.3 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [7]:
def squared_num(a):
    return a**2


def square_and_select(ind, array):
    return array[ind] ** 2


def ex_ante_select(indices, array_to_select):
    selected_array = jnp.take(array_to_select, indices)
    return vmap(squared_num, in_axes=(0))(selected_array)


def select_on_the_go(indices, array_to_select):
    return vmap(square_and_select, in_axes=(0, None))(indices, array_to_select)

In [3]:
size_ind = 100_000_000
to_select_test = jnp.array([2, 4])
indices_test = jnp.append(jnp.zeros(size_ind), jnp.ones(size_ind)).astype(int)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [8]:
jit_exante = jit(ex_ante_select)
jit_expost = jit(select_on_the_go)

In [5]:
jit_exante(indices_test, to_select_test)
%timeit jit_exante(indices_test, to_select_test)

154 ms ± 2.34 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
jit_expost(indices_test, to_select_test)
%timeit jit_expost(indices_test, to_select_test)

156 ms ± 2.15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
